In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from helper import load_data
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

/root/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 150
CHANNELS = 3
NUM_EPOCH = 30
LEARN_RATE = 1.0e-4

The question raised is why rescale is 1./255 and why we need this before training neural network. For colorful image, every image contains three maps: Red, Green and Blue, and all the pixel still in the range 0~255.
Rescale is important to treat all images in the same manner: some images are high pixel range, some are low pixel range. Scaling every images to the same range [0,1] will make images contributes more evenly to the total loss. 

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255.)
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=32,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        'data/valid',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=32,
        class_mode='categorical')

test_generator = train_datagen.flow_from_directory(
        'data/test',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [4]:
def cnn_model():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS)))    
    model.add(Conv2D(32, (3, 3), activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(3, activation='softmax'))
    
    model.summary()
    
    return model

In [5]:
model = cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
__________

In [6]:
model.load_weights('best_model_simple.h5')

In [9]:
from PIL import Image
import numpy as np
from keras.preprocessing import image
import csv
cnt = 0

with open('submission.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    filewriter.writerow(['Id', 'task_1', 'task_2'])

    for i, n in enumerate(test_generator.filenames):

        path = 'data/test/' + n
        img = Image.open(path)
        img = image.array_to_img(img, scale=False)
        img = img.resize((150, 150))
        img = image.img_to_array(img)


        img = np.array(img).reshape(1, 150, 150,3)
        pred =  model.predict(img, batch_size=32, verbose=0)[0]
        filewriter.writerow([n, pred[0], pred[2]])


        cnt += 1